In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [ ]:
n = 55000
mcl = 10
dataset = f"sine_rw_{mcl}_1234"

In [ ]:
from data.rw_generator import *

In [ ]:
dfs = []
rw = []

In [ ]:
rw.append(RandomWalkGenerator(generator = 'sine', classification_function=0))
dfs.append(rw[-1].generate(n, max_consec_labels = mcl, distribution="chi2"))

In [ ]:
rw.append(RandomWalkGenerator(generator = 'sine', classification_function=2))
dfs.append(rw[-1].generate(n, max_consec_labels = mcl, distribution="normal"))

In [ ]:
rw.append(RandomWalkGenerator(generator = 'sine', classification_function=1))
dfs.append(rw[-1].generate(n, max_consec_labels = mcl, distribution="uniform"))

In [ ]:
rw.append(RandomWalkGenerator(generator = 'sine', classification_function=3))
dfs.append(rw[-1].generate(n, max_consec_labels = mcl, distribution="f"))

In [ ]:
import pickle
with open(f"datasets/{dataset}.pkl", "wb") as f:
    pickle.dump(rw, f)

In [ ]:
dfs_train = []
dfs_test = []
for i in range(0,len(dfs)):
    dfs[i]["task"]=i+1
    dfs_train.append(dfs[i].iloc[:-5000,0:])
    dfs_test.append(dfs[i].iloc[-5000:,0:])

In [ ]:
df_train = pd.concat(dfs_train)
df_test = pd.concat(dfs_test)

In [ ]:
df_train.to_csv(f"datasets/{dataset}.csv", index=False)
df_test.to_csv(f"datasets/{dataset}_test.csv", index=False)

In [ ]:
import sklearn
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=len(dfs), figsize=(30, 30))
for i,df in enumerate(dfs):
    df_ = df.reset_index().drop(columns="index").iloc[0:3000,0:]
    ax[i].plot(df_.index, df_["target"])
    ax[i].scatter(df_.index, df_["target"])

In [ ]:
dfs = dfs_train

In [ ]:
lens = []
for t in range(0, df_train["task"].max()):
    i = 1
    j = 0
    lens.append([[],[]])
    
    while i < len(dfs[t]):
        if dfs[t].iloc[i]["target"] != dfs[t].iloc[i-1]["target"]:
            lens[-1][int(dfs[t].iloc[i-1]["target"])].append(i-j)
            j = i
        i += 1
    lens[-1][int(dfs[t].iloc[i-1]["target"])].append(len(dfs[t])-j)

In [ ]:
from collections import Counter

counters = []

for i in range(0,len(lens)):
    counters.append([])
    for j in range(0,len(lens[i])):
        c = dict(Counter(lens[i][j]))
        for k in range(1, 21):
            if k not in c:
                c[k] = 0
        ordered_c = []
        for k in sorted(c.keys()):
            ordered_c.append(c[k])
        counters[-1].append(ordered_c)

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=len(dfs), figsize=(30, 30), sharey=True)
for i, df_ in enumerate(dfs):
    ax[i].bar([str(x) for x in list(range(1,len(counters[i][0])+1))], counters[i][0], 1, label="0")
    ax[i].bar([str(x) for x in list(range(1,len(counters[i][1])+1))], counters[i][1], 1, label="1", bottom = counters[i][0])
    ax[i].legend()

In [ ]:
def create_inverted(dfs):
    dfs_inverted = []
    for t, i in enumerate([0, 3, 2, 1]):
        dfs_inverted.append(dfs[i].copy())
        dfs_inverted[-1]["task"] = t+1
    return pd.concat(dfs_inverted)

In [ ]:
df_train_inverted = create_inverted(dfs_train)
df_test_inverted = create_inverted(dfs_test)

In [ ]:
df_train_inverted.to_csv(f"datasets/{dataset}_inverted.csv", index=False)
df_test_inverted.to_csv(f"datasets/{dataset}_test_inverted.csv", index=False)